### Basic Agent & Tool

### 1) **Agent + Runner + Tools**

**원리/프로세스**

* `@function_tool` 데코레이터로 파이썬 함수를 **툴**로 노출
  - 이때 **함수 시그니처와 독스트링**을 바탕으로 툴 스키마가 자동 생성  
  - `get_weather(city: str)`는 입력 도시와 무관하게 데모 문자열 `"30 degrees"`를 반환하는 **모의(Mock) 툴**
* `Agent(...)`에 `tools=[get_weather]`로 툴을 연결하면, 모델이 필요하다고 판단할 때 **툴 호출 이벤트**를 발생
* `Runner.run_streamed(agent, prompt)`는 **스트리밍 핸들러**를 즉시 반환(여기서는 `stream`)
  - 이후 `stream.stream_events()`를 `async for`로 순회하며 **실시간 이벤트**를 소비

* 세 가지 이벤트 처리

  * `raw_response_event` : **하위 레벨 응답 이벤트**(토큰/델타 등)
    - 여기서는 **PASS**
  * `agent_updated_stream_event` : 실행 중 **에이전트 상태가 바뀔 때**(핸드오프) 발생
    - `event.new_agent.name`을 출력
  * `run_item_stream_event` : 실행 항목 단위의 이벤트

    * `tool_call_item` : 모델이 **툴을 호출하려는 의도**를 생성했을 때의 항목(원본 호출 내용을 `to_dict()`로 출력)
    * `tool_call_output_item` : **툴 실행 결과**(파이썬 함수의 반환값)
    * `message_output_item` : 모델이 생성한 **최종 메시지** 항목이며, `ItemHelpers.text_message_output(...)`으로 텍스트를 추출해 출력

**작동 관찰 포인트**

* 프롬프트가 단순 인사(예: `"Hello, how are you?"`)라면, 보통 **툴 호출 없이** `message_output_item`만 출력될 수 있음
    - 툴 호출 흐름을 보고 싶다면 “부산/서울 날씨 알려줘”처럼 **툴을 필요로 하는 요청** 실행
* `raw_response_event`를 건너뛰고 `run_item_stream_event`만 보이도록 해 **고수준 이벤트만** 확인하는 패턴

**개념 정리**

* **툴 호출 흐름**: `tool_call_item`(호출 의도) → SDK가 실제 함수 실행 → `tool_call_output_item`(반환) → 모델이 결과를 사용해 최종 `message_output_item` 생성
* **스트리밍**: “최종 완성본”을 기다리지 않고, **이벤트 단위**로 실행 과정을 관찰/렌더링 가능
* **헬퍼**: `ItemHelpers.text_message_output(...)`은 메시지 항목에서 **사람이 읽을 수 있는 텍스트**만 간단히 추출

In [ ]:
# 모듈 임포트
from agents import Agent, Runner, function_tool, ItemHelpers

# 함수 툴로 데코레이터
@function_tool
def get_weather(city: str):
    """Get weather by City"""
    return "30 degrees"

# 에이전트 생성 (이름/지시문/모델/도구 설정)
agent = Agent(
    # 변수/객체 할당
    name="Assistant Agent",

    # 에이전트 지시문 설정
    instructions="You are a helpful assistant. Use tools when needed to answer questions.",
    
    # 에이전트에 툴 연결
    tools=[get_weather]
)

# 스트리밍 실행 핸들러 생성
stream = Runner.run_streamed(agent, "Hello, how are you?")

# 스트리밍 이벤트 비동기 순회
async for event in stream.stream_events():
    if event.type == "raw_response_event":
        continue # 다음 루프
    elif event.type == "agent_updated_stream_event":
        print("Agent updated to", event.new_agent.name)
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print(event.item.raw_item.to_dict())
        elif event.item.type == "tool_call_output_item":
            print(event.item.output)
        elif event.item.type == "message_output_item":
            print(ItemHelpers.text_message_output(event.item))

### 2) **이벤트 스트림**

**원리/프로세스**
* `Runner.run_streamed(agent, prompt)`로 스트리밍 핸들러 획득

  * 반환값은 즉시 사용 가능한 스트림 객체
  * `stream.stream_events()`를 `async for`로 순회해 **원시 응답 이벤트** 소비
* `raw_response_event` 분기에서 **내부 이벤트 타입**(`event.data.type`) 확인

  * `response.output_text.delta` → 출력 텍스트 델타 누적
  * `response.function_call_arguments.delta` → 툴 인자(JSON) 델타 누적
  * `response.completed` → 한 턴 종료 신호로 누적 버퍼 초기화

**이벤트 처리 흐름**

* 바깥 이벤트 선택

  * `event.type == "raw_response_event"` 인 경우만 내부 타입 분기
* 내부 타입 분기

  * `response.output_text.delta` → `message += event.data.delta`
  * `response.function_call_arguments.delta` → `args += event.data.delta`
  * `response.completed` → `message = ""`, `args = ""` 초기화

**작동 관찰 포인트**

* 프롬프트가 일반 인사면 텍스트 델타만 도착할 가능성 높음

  * 툴이 필요해 보이는 질문 사용-> **툴 인자 델타** 출력
* `response.completed` 이후 버퍼를 비워 다음 응답과 구분

**주의**

* 코드 내 `arge` 변수는 `args`의 오타로 보임

  * 구조 변경 없이 설명만 제공
* `ItemHelpers`는 이 셀에서 직접 사용하지 않음

  * 상단 임포트는 이전/다음 셀 재사용을 위한 것으로 해석


In [ ]:
# 모듈 임포트
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city: str):
    """Get weather by City"""
    return "30 degrees"

# 에이전트 생성 (이름/지시문/모델/도구 설정)
agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions.",
    tools=[get_weather]
)

# 스트리밍 실행 핸들러 생성
stream = Runner.run_streamed(agent, "Hello, how are you?")

message = ""
args = ""

# 스트리밍 이벤트 비동기 순회
async for event in stream.stream_events():
    if event.type == "raw_response_event":
        event_type = event.data.type
        if event_type == "response.output_text.delta":
            message += event.data.delta
            print(message)
        elif event_type == "response.function_call_arguments.delta":
            arge += event.data.delta
            print(args)
        elif event_type == "response.completed":
            message = ""
            args = ""

### 3) **세션 메모리(SQLiteSession)로 맥락 유지**

**원리/프로세스**

* `SQLiteSession(session_id, db_path)`로 대화 히스토리 저장소 생성

  * 동일한 `session_id`로 여러 턴을 실행하면 과거 발화가 컨텍스트로 재사용
* `Runner.run(..., session=session)`으로 세션을 넘겨주면 이전 턴 메시지가 모델 컨텍스트에 자동 주입
* 이 셀은 과거에 **이 세션으로 이름을 말해둔 턴이 있을 때만** “내 이름”을 기억

  * 이 셀만 단독 실행하면 이름 정보 없음

**흐름**

1. 세션 초기화 → 2) 에이전트와 툴 준비 → 3) 세션을 포함해 한 턴 실행 → 4) `final_output` 확인

**작동 관찰 포인트**

* 같은 노트북에서 **바로 앞 셀**에서 `session="user_2"`로 “내 이름은 OOO” 같은 입력을 먼저 남겨두면 효과 확인
* 세션 DB 파일 `ai-memory.db`가 생성되어 지속 저장
* 세션을 바꾸면 완전히 새로운 대화로 취급

**주의**

* 수업 환경에서 여러 명이 같은 경로를 공유하면 세션ID 충돌 가능

  * `user_2` 대신 실습자 고유 ID 지정 권장
* 개인정보를 세션에 남길 때는 파일 경로 관리 필요

In [ ]:
from agents import Agent, Runner, function_tool, SQLiteSession

session = SQLiteSession("user_2", "ai-memory.db")

@function_tool
def get_weather(city: str):
    """Get weather by City"""
    return "30 degrees"

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions.",
    tools=[get_weather]
)

In [ ]:
result = await Runner.run(
    agent, 
    "What was my name again?",
    session=session,
)

print(result.final_output)

Your name is Seojun.


In [ ]:
await session.clear_session()

In [ ]:
await session.add_items(
    [{"role": "user", "content": "My name is Seojun"}]
)

In [ ]:
await session.pop_item()

### 4) **Handoffs — 분기 에이전트에서 전문가 에이전트로 라우팅**

**원리/프로세스**

* 여러 전문 에이전트를 `handoffs=[...]`로 등록

  * 각 에이전트의 `handoff_description`이 라우팅 근거
* 메인 에이전트가 사용자의 질문을 해석해 **가장 적합한 전문가**로 핸드오프
* 세션이 있으면 라우팅 결정에도 과거 맥락이 영향을 줄 수 있음

**흐름**

1. 세션 준비 → 2) 지리 전문가, 경제 전문가 정의 → 3) 메인 에이전트에 두 전문가를 `handoffs`로 연결  
   → 4) `Runner.run(main_agent, question, session=...)` 실행 → 5) `result.last_agent`로 실제 라우팅 결과 확인

**작동 관찰 포인트**

* 질문이 “국가가 왜 국채를 발행하나?”이면 **경제 전문가**로 라우팅 기대
* 동일 질문이라도 지시문·설명 품질에 따라 라우팅 결과가 달라질 수 있음

  * `instructions`와 `handoff_description`을 짧고 명확하게 유지
* `result.final_output`은 최종 응답, `result.last_agent.name`은 마지막으로 응답한 에이전트 이름

**주의**

* `handoff_description`은 **전문 분야 한 줄 요약**에 집중
* 전문가 수가 늘면 모호성 증가

  * 메인 에이전트 `instructions`에 간단한 규칙 예시를 넣어 힌트 제공
* 세션을 바꾸면 라우팅 판단도 바뀔 수 있음

In [ ]:
from agents import Agent, Runner, SQLiteSession

session = SQLiteSession("user_1", "ai-memory.db")

geography_agent = Agent(
    name="Geo Expert Agent",
    instructions="You are an expert in geography, you answer questions related to them.",
    handoff_description="Use this to answer geography related questions"
)

economics_agent = Agent(
    name="Economics Expert Agent",
    instructions="You are an expert in economics, you answer questions related to them.",
    handoff_description="Use this to answer economic related questions"
)


main_agent = Agent(
    name="Main Agent",
    instructions="You are a user facing agent. Transfer to the agent most capable of answering the user's question.",
    handoffs=[
        economics_agent,
        geography_agent,
    ]
)

In [ ]:
result = await Runner.run(
    main_agent, 
    "Why do countries sell bonds?",
    session=session,
)

print(result.last_agent.name)
print(result.final_output)

### 5) **시각화 + 구조화 출력 + 실행**

**원리/프로세스**

* `output_type=Answer`로 **Pydantic 스키마** 유도

  * 최종 응답을 `{"answer": str, "background_explanation": str}` 형태로 수렴
* `@function_tool`로 툴 노출

  * `get_weather()`는 모의(Mock) 툴. 반환 `"30"`
* `draw_graph(main_agent)`로 에이전트 구성 **시각화**

  * 메인 → 지리/경제 전문가로 **핸드오프 구조** 확인
* `Runner.run(...)`으로 한 턴 **실행**

  * `session`을 넘겨 과거 컨텍스트 재사용
  * `result.last_agent.name`으로 실제 라우팅 대상 확인

**흐름**

1. 세션 생성 → 2) `Answer` 스키마 정의 → 3) 지리/경제 에이전트 준비   
  → 4) 메인 에이전트에 `handoffs`와 `output_type` 설정 → 5) 그래프 시각화 → 6) 한 턴 실행 → 7) 라우팅 대상·최종 출력 확인

**작동 관찰 포인트**

* 지리 질문이면 `Geo Expert Agent` 라우팅 기대
* `economics_agent`에 `output_type` 미지정 → 라우팅이 경제로 가면 형식 불일치 가능성
* 질의에 오탈자(`northen`) 포함 → 모델이 자동 보정할 가능성

**주의**

* 구조화 일관성 확보 목적이면 **모든 최종 후보 에이전트**에 동일 `output_type` 지정 권장
* `trace` 임포트는 이 코드에서 직접 사용하지 않음

In [ ]:
from agents import Agent, Runner, SQLiteSession, function_tool, trace
from agents.extensions.visualization import draw_graph
from pydantic import BaseModel


session = SQLiteSession("user_1", "ai-memory.db")

class Answer(BaseModel):
    answer: str
    background_explanation: str



@function_tool
def get_weather():
    return "30"


geography_agent = Agent(
    name="Geo Expert Agent",
    instructions="You are an expert in geography, you answer questions related to them.",
    handoff_description="Use this to answer geography related questions",
    tools=[
        get_weather,
    ],
    output_type=Answer
)

economics_agent = Agent(
    name="Economics Expert Agent",
    instructions="You are an expert in economics, you answer questions related to them.",
    handoff_description="Use this to answer economic related questions"
)


main_agent = Agent(
    name="Main Agent",
    instructions="You are a user facing agent. Transfer to the agent most capable of answering the user's question.",
    handoffs=[
        economics_agent,
        geography_agent,
    ],
    output_type=Answer
)

draw_graph(main_agent)

In [ ]:
result = await Runner.run(
main_agent, 
"What is the capital of Thailand's northen province",
session=session,
)

print(result.last_agent.name)
print(result.final_output)

### 6) **Trace 컨텍스트로 실행 흐름 묶기**

**원리/프로세스**

* `with trace("user_3"):` 컨텍스트 내부의 **여러 번 실행**을 하나의 트레이스 세션으로 묶음

  * 대시보드 Traces에서 **같은 그룹**으로 조회
  * 프롬프트·핸드오프·툴 호출·토큰 타임라인을 한눈에 비교
* 동일 `session` 사용으로 대화 맥락 공유

  * 라우팅·응답이 이전 턴의 영향 받음

**흐름**

1. 에이전트 구성 시각화로 구조 확인
2. `trace("user_3")` 블록 시작
3. `Runner.run(main_agent, ...)` 세 번 호출
4. 트레이스 종료 후 대시보드에서 타임라인 비교

**작동 관찰 포인트**

* 세 질의 모두 `"northen"` 오탈자 포함

  * 모델이 자동 보정 시도 가능
  * 지리 질문 성격 → `Geo Expert Agent` 라우팅 기대
* 각 턴의 툴 호출 여부, 최종 구조화 출력 일치 여부 확인
* 세션 유지에 따른 누적 효과 확인

**주의**

* 트레이스 라벨 `"user_3"`는 임의 식별자

  * 실습자별·세션별로 구분 라벨 사용 권장
* 구조화 일관성을 원하면 모든 후보 에이전트에 `output_type` 지정

In [ ]:
from agents import Agent, Runner, SQLiteSession, function_tool, trace
from agents.extensions.visualization import draw_graph
from pydantic import BaseModel


session = SQLiteSession("user_1", "ai-memory.db")

class Answer(BaseModel):
    answer: str
    background_explanation: str



@function_tool
def get_weather():
    return "30"


geography_agent = Agent(
    name="Geo Expert Agent",
    instructions="You are an expert in geography, you answer questions related to them.",
    handoff_description="Use this to answer geography related questions",
    tools=[
        get_weather,
    ],
    output_type=Answer
)

economics_agent = Agent(
    name="Economics Expert Agent",
    instructions="You are an expert in economics, you answer questions related to them.",
    handoff_description="Use this to answer economic related questions"
)


main_agent = Agent(
    name="Main Agent",
    instructions="You are a user facing agent. Transfer to the agent most capable of answering the user's question.",
    handoffs=[
        economics_agent,
        geography_agent,
    ],
    output_type=Answer
)

draw_graph(main_agent)

In [ ]:
with trace("user_3"):
    result = await Runner.run(
        main_agent, 
        "What is the capital of Colombia's northen province",
        session=session,
    )

    result = await Runner.run(
        main_agent, 
        "What is the capital of Cambodia's northen province",
        session=session,
    )

    result = await Runner.run(
        main_agent, 
        "What is the capital of Thailand's northen province",
        session=session,
    )
    